# # Loading Data To Gold Zone 

**This Notebook:**
* Load data to Golg Zone of the Data Lake House
* Star Schekma and One Big Table Modeling
* Creates **`IDENTITY`** column in Databricks delta table

## 1.0 Initial Setup

In [0]:

%run "/Users/cabreirajm@gmail.com/DataPipelineCabreira/Helpers/data_generator" 

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
  Using cached dbldatagen-0.4.0.post1-py3-none-any.whl (122 kB)
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


## 2.0 Create `Gold Zone` Schema

In [0]:
spark.sql("CREATE DATABASE IF NOT EXISTS gold")

DataFrame[]

## 3.0 `Sales Star Schema` Modeling 

Aiming to optimize queries in large datasets, we can use a dimensional model. 
We will use Ralph Kimball data warehouse principles and build a Star Schema model.


### `Dimensional Tables`
- **dim_calendar** - Dimension with date information
- **dim_cod** - Dimensions with codes  - Low cardinality Dimensions (Junk Dimension): 
  - **user_origin** - API vs. Files
  - **access_from** - mobile vs. computer
  - **payment_method** - Pix vs. Boleto vs. Cartão
  - **percent_discount** - 5% vs. 10% vs. 15%
- **dim_courses** - Dimensão responsável por armazenar as informações de Curso.
- **dim_user** - Dimensão responsável por armazenar as informações de Alunos.


All tables will have a **Surrogate Key (SK)** column that will be creeated with the **`<col_name> BIGINT GENERATED ALWAYS AS IDENTITY`** command. Spark will populate this column in execution time with an incremental value (incremental(1,1). )


### 3.1 `Sale Dimensions`  

In [0]:
#%sql
#DROP SCHEMA IF EXISTS gold CASCADE 


In [0]:
%fs rm -r dbfs:/user/hive/warehouse/gold.db/dim_calendar

res16: Boolean = false

In [0]:

%fs rm -r dbfs:/user/hive/warehouse/gold.db/dim_cod

res17: Boolean = false

In [0]:
%fs rm -r dbfs:/user/hive/warehouse/gold.db/dim_course

res18: Boolean = false

In [0]:
%fs rm -r dbfs:/user/hive/warehouse/gold.db/dim_user

res19: Boolean = false

In [0]:
spark.sql("""
    CREATE TABLE IF NOT EXISTS gold.dim_calendar(
        sk_tempo BIGINT GENERATED ALWAYS AS IDENTITY,
        date DATE,
        year INT, 
        month STRING,
        month_year INT,
        day_week_int INT, 
        day_week STRING,
        fl_day_week BOOLEAN,
        day_month INT,
        fl_last_month_day INT,
        day_year INT,
        week_year INT,
        bimonthly INT,
        quarter INT, 
        semester INT, 
        dt_load TIMESTAMP
    )  
""")

spark.sql("""
    CREATE TABLE IF NOT EXISTS gold.dim_cod (
        sk_cod BIGINT GENERATED ALWAYS AS IDENTITY,
        user_origin STRING,
        access_from STRING,
        payment_method STRING,
        percent_discount STRING,
        dt_load TIMESTAMP
    )  
""")

spark.sql("""
    CREATE TABLE IF NOT EXISTS gold.dim_course(
        sk_course BIGINT GENERATED ALWAYS AS IDENTITY,
        course_uuid STRING,
        course_name STRING, 
        course_level STRING,
        course_price DECIMAL(9,2),
        dt_load TIMESTAMP
    )          
""")

spark.sql("""
    CREATE TABLE IF NOT EXISTS gold.dim_user(
    sk_user BIGINT GENERATED ALWAYS AS IDENTITY,
    user_uuid STRING,
    user_name STRING,
    user_email STRING,
    user_idade INT, 
    user_gender STRING,
    user_state STRING,
    user_profession STRING,
    company STRING,
    dt_load TIMESTAMP
    )
""")

DataFrame[]

### 3.2 `Calendar Dimension`  

The view **`vw_dim_calendar`**:
* Starts date : **01/06/2024** 
* End date: **31/12/2025**

In [0]:

from pyspark.sql.functions import explode, sequence, to_date

start_date = '2024-06-01'
end_date = '2025-12-31'

spark.sql(f"""         
  with dates as (
    select
      explode(
        sequence(
          to_date('{start_date}'),
          to_date('{end_date}'),
          interval 1 day
        )
      ) as date
  )
  select
    date,
    year(date) AS year,
    to_csv(
      named_struct('date', date),
      map('dateFormat', 'MMMM', 'locale', 'EN')
    ) AS month,
    month(date) as month_year,
    dayofweek(date) AS day_week_int,
    to_csv(
      named_struct('date', date),
      map('dateFormat', 'EEEE', 'locale', 'EN')
    ) AS day_week,
    case
      when weekday(date) < 5 then True
      else False
    end as fl_day_week,
    dayofmonth(date) as day_month,
    case
      when date = last_day(date) then True
      else False
    end as fl_last_month_day,
    dayofyear(date) as day_year,
    weekofyear(date) as week_year,
    case
      when month(date) in (1, 2) then 1
      when month(date) in (3, 4) then 2
      when month(date) in (5, 6) then 3
      when month(date) in (7, 8) then 4
      when month(date) in (9, 10) then 5
      when month(date) in (11, 12) then 6
    end as bimonthly,
    case
      when month(date) in (1, 2, 3) then 1
      when month(date) in (4, 5, 6) then 2
      when month(date) in (7, 8, 9) then 3
      when month(date) in (10, 11, 12) then 4
    end as quarter,
    case
      when month(date) in (1, 2, 3, 4, 5, 6) then 1
      when month(date) in (7, 8, 9, 10, 11, 12) then 2
    end as semester
  from
    dates
""").createOrReplaceTempView('vw_dim_calendar')

spark.sql('SELECT * FROM vw_dim_calendar LIMIT 3').display()

date,year,month,month_year,day_week_int,day_week,fl_day_week,day_month,fl_last_month_day,day_year,week_year,bimonthly,quarter,semester
2024-06-01,2024,June,6,7,Saturday,false,1,false,153,22,3,2,1
2024-06-02,2024,June,6,1,Sunday,false,2,false,154,22,3,2,1
2024-06-03,2024,June,6,2,Monday,true,3,false,155,23,3,2,1



We will now use the **`Merge`** commando to load the `dim_calendar` data.

In [0]:
spark.sql("""
    MERGE INTO gold.dim_calendar as dest
    USING vw_dim_calendar AS orig
        ON dest.date = orig.date
    WHEN NOT MATCHED    
        THEN INSERT(
            date,year,month,month_year,day_week_int,day_week,fl_day_week,day_month,fl_last_month_day,day_year,week_year,bimonthly,quarter,semester,dt_load
            )
        VALUES(
            date,year,month,month_year,day_week_int,day_week,fl_day_week,day_month,fl_last_month_day,day_year,week_year,bimonthly,quarter,semester,getdate()
        )
    """).display()


spark.sql("SELECT * FROM gold.dim_calendar LIMIT 5").display()

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
579,0,0,579


sk_tempo,date,year,month,month_year,day_week_int,day_week,fl_day_week,day_month,fl_last_month_day,day_year,week_year,bimonthly,quarter,semester,dt_load
1,2024-06-01,2024,June,6,7,Saturday,false,1,0,153,22,3,2,1,2024-11-28T01:32:25.472Z
2,2024-06-02,2024,June,6,1,Sunday,false,2,0,154,22,3,2,1,2024-11-28T01:32:25.472Z
3,2024-06-03,2024,June,6,2,Monday,true,3,0,155,23,3,2,1,2024-11-28T01:32:25.472Z
4,2024-06-04,2024,June,6,3,Tuesday,true,4,0,156,23,3,2,1,2024-11-28T01:32:25.472Z
5,2024-06-05,2024,June,6,4,Wednesday,true,5,0,157,23,3,2,1,2024-11-28T01:32:25.472Z


## 3.3 Junk Dimension
We will first create a **`vw_dim_cod`** view that will be responsible for creating a **cartesian product between some of low cardinality codes***. 

We will use this method in order to avoid building low dimensions with few registers.


In [0]:
spark.sql( """
    SELECT DISTINCT 
    s.origin user_origin ,
    CASE WHEN s.origin = 'File' then "do not apply" ELSE a.local_access end as local_access,
    s.payment_method,
    coalesce(s.percent_discount, 'do not apply' ) as percent_discount
    from silver.tb_sales as s
    cross join silver.tb_access as a 
    order BY user_origin,local_access, payment_method, percent_discount      
""").createOrReplaceTempView("vw_dim_cod")

spark.sql("select * from vw_dim_cod limit 5").display()

user_origin,local_access,payment_method,percent_discount
API,Computer,boleto,5%
API,Computer,boleto,do not apply
API,Computer,credito,10%
API,Computer,credito,15%
API,Computer,credito,5%


In [0]:
spark.sql("""
    MERGE INTO gold.dim_cod as dest
    using vw_dim_cod as orig
    on dest.user_origin = orig.user_origin
        and dest.payment_method = orig.payment_method
        and dest.access_from = orig.local_access
        and dest.percent_discount = orig.percent_discount

    when not matched 
        then INSERT (
            user_origin,
            access_from,
            payment_method,
            percent_discount,
           dt_load
        )
        values(
            user_origin,
            local_access,
            payment_method,
            percent_discount,
           getdate()
        )       
""").display()

spark.sql("SELECT * FROM gold.dim_cod LIMIT 5").display()

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
22,0,0,22


sk_cod,user_origin,access_from,payment_method,percent_discount,dt_load
1,API,Computer,boleto,5%,2024-11-28T01:32:35.359Z
2,API,Computer,boleto,do not apply,2024-11-28T01:32:35.359Z
3,API,Computer,credito,10%,2024-11-28T01:32:35.359Z
4,API,Computer,credito,15%,2024-11-28T01:32:35.359Z
5,API,Computer,credito,5%,2024-11-28T01:32:35.359Z


## 3.4 User Dimension

In [0]:
spark.sql("""
      MERGE INTO gold.dim_user as dest 
      using silver.tb_users as orig
        on dest.user_uuid = orig.user_uuid
    when matched
        and dest.user_email != orig.user_email
        or dest.user_idade!= orig.user_idade
        or dest.user_state != orig.user_state
        or dest.user_profession != orig.user_profession
        or dest.company != orig.company
        THEN UPDATE 
            SET dest.user_email = orig.user_email
                ,dest.user_idade = orig.user_idade 
                ,dest.user_state = orig.user_state
                ,dest.user_profession = orig.user_profession
                ,dest.company  = orig.company
          
         when not MATCHED
            then insert (
        user_uuid, 
        user_name, 
        user_email, 
        user_idade, 
        user_gender,
        user_state, 
        user_profession, 
        company, 
        dt_load
            )
    VALUES (
        user_uuid, 
        user_name, 
        user_email, 
        user_idade, 
        user_gender,
        user_state, 
        user_profession, 
        company, 
        getdate()

            )   
""").display()

spark.sql("SELECT * FROM gold.dim_user LIMIT 5").display()

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
157185,0,0,157185


sk_user,user_uuid,user_name,user_email,user_idade,user_gender,user_state,user_profession,company,dt_load
1,8d1a271791b6a3fb5af5c34031eaae4f,Usuario 5250b1b2f4,usuario_5250b1b2f4@uol.com,32,F,PE,Analista de BI,null,2024-11-28T01:32:45.471Z
8,2be5820316b8dfad2d6d5f447eec1e71,Usuario 551b70427a,usuario_551b70427a@uol.com,27,M,PR,Analista de Dados,null,2024-11-28T01:32:45.471Z
15,5d7cc1629ba8d330729524fc8e8fb86e,Usuario 1c84387f7d,usuario_1c84387f7d@hotmail.com,32,F,AC,Arquiteto de Dados,null,2024-11-28T01:32:45.471Z
22,5c44a8122ab24c978b35c79548f72a43,Usuario 2935de7d69,usuario_2935de7d69@uol.com,26,M,TO,Engenheiro de Dados,null,2024-11-28T01:32:45.471Z
29,a550627644f51e197c94935ad31de86f,Usuario 528ec8140b,usuario_528ec8140b@uol.com,42,F,SE,Cientista de Dados,null,2024-11-28T01:32:45.471Z


## 3.5 Courses Dimension

We will load the **`dim_course`** with  **`MERGE`**  command
* Source: **`silver.tb_curso`**. table


In [0]:
spark.sql("""
    merge into gold.dim_course as dest 
        using silver.tb_courses as orig
            on dest.course_uuid = orig.course_uuid

    when matched    
        and dest.course_name != orig.course_name
        or dest.course_level != orig.course_level
        or dest.course_price != orig.course_price
        then update 
            set 
                dest.course_name = orig.course_name,
                dest.course_level = orig.course_level,
                dest.course_price = orig.course_price
    when not matched
        then insert (
            course_uuid,
            course_name,
            course_level,
            course_price,
            dt_load
        )
    values (
            course_uuid,
            course_name,
            course_level,
            course_price,
            GETDATE()
    )
""").display()

spark.sql("SELECT * FROM gold.dim_course LIMIT 5").display()

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
3,0,0,3


sk_course,course_uuid,course_name,course_level,course_price,dt_load
1,fb95df132ca7f41d392bc98ccf0cfeb8,Data Pipeline with Databricks,beginner,589.90,2024-11-28T01:32:54.016Z
2,bda125b01c9596e123e5f9b3bf00f3a8,From your first data pipeline to a Data Lakehouse with Databricks,intermediate,659.90,2024-11-28T01:32:54.016Z
3,ff17869bc6f9d9865e0bf8133c4ce3c3,Building a Data Pipeline with Spark Structured Streaming,advanced,549.90,2024-11-28T01:32:54.016Z


## 3.6 Sales Fact

In [0]:
df_stream_sales= spark.readStream.table('silver.tb_sales')
df_stream_sales.createOrReplaceTempView('vw_fact_sales')

In [0]:
df_fact_sales = spark.sql("""
    with orig_fact_sales (
        select 
            s.percent_discount,
            case when s.origin = 'FILE' then 'not applicable' else a.local_access end as access_local,
            s.payment_method,
            s.origin as user_origin,
            s.discount_value,
            s.total_value,
            s.course_uuid,
            s.dt_sale
        from vw_fact_sales as s
        left join silver.tb_access as a on s.access_uuid = a.access_uuid
    )    
    select   
        dcal.sk_tempo,
        dcod.sk_cod,
        dc.sk_course,
        du.sk_user,
        ofs.discount_value,
        ofs.total_value
    from orig_fact_sales as ofs
    join gold.dim_cod as dcod on ofs.payment_method = dcod.payment_method
                                and ofs.user_origin = dcod.user_origin
                                and ofs.access_local =dcod.access_from
                                and ofs.percent_discount = dcod.percent_discount
    join gold.dim_calendar as dcal on dcal.date = cast(ofs.dt_sale as date)
    join gold.dim_course as dc on dc.course_uuid = ofs.course_uuid
    join gold.dim_user as du on du.user_uuid = ofs.user_uuid

""")

df_fact_sales.limit(5).display()

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-3337697916647392>, line 1
----> 1 df_fact_sales = spark.sql("""
      2     with orig_fact_sales (
      3         select 
      4             s.percent_discount,
      5             case when s.origin = 'FILE' then 'not applicable' else a.local_access end as access_local,
      6             s.payment_method,
      7             s.origin as user_origin,
      8             s.discount_value,
      9             s.total_value,
     10             s.course_uuid,
     11             s.dt_sale
     12         from vw_fact_sales as s
     13         left join silver.tb_access as a on s.access_uuid = a.access_uuid
     14     )    
     15     select   
     16         dcal.sk_tempo,
     17         dcod.sk_cod,
     18         dc.sk_course,
     19         du.sk_user,
     20         ofs.discount_value,
     21         ofs.tota